# Draft Health Inspections Dashboard

This notebook is a place to toy with some interactive widgets that might be part of a tutorial.

In [ ]:
import pandas as pd
import numpy as np
import json
import string
import io
import requests
import declarativewidgets

In [ ]:
declarativewidgets.init()

Individual DataFrames.

In [ ]:
est = pd.read_csv('https://ibm.box.com/shared/static/7r91a7y5qk47au8wsuwvvfp38p1ufoo7.csv', parse_dates=['opened', 'closed'])

In [ ]:
ins = pd.read_csv('https://ibm.box.com/shared/static/t69z2rbvzmrrtlqzlfearhx46hpj2uur.csv', parse_dates=['date'])

In [ ]:
vio = pd.read_csv('https://ibm.box.com/shared/static/cnixvlrt43qv56f2docll423rkau0oet.csv')

In [ ]:
est['latlon'] = est.latlon.str.split(', ')

Joined DataFrames.

In [ ]:
est_ins = pd.merge(est, ins, left_on='id', right_on='est_id', suffixes=('_est', '_ins'))

In [ ]:
est_ins_vio = pd.merge(est_ins, vio, left_on='id_ins', right_on='insp_id', suffixes=('', '_vio'))

## Filter By Name

In [ ]:
%%html
<link rel="import" href="urth_components/paper-input/paper-input.html" is='urth-core-import'>

In [ ]:
%%html
<template is="urth-core-bind">
    <paper-input value="{{pattern}}" label="Filter by establishment name" ></paper-input>
</template>

In [ ]:
def filter_df(df, pattern):
    return df[df.name.str.contains(pattern, case=False)]

## Table of Establishments

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is='urth-core-import'>

In [ ]:
def est_table(pattern=''):
    '''
    Build a DataFrame with basic info about establishments.
    Include information from the latest inspection.
    Clean up the names a bit and format the dates.
    '''
    df = (filter_df(est_ins, pattern)
          .dropna(subset=['final_score'])
          .sort_values(['name', 'date'])
          .drop_duplicates('id_est', keep='last'))
    df['name'] = df.name.str.slice(stop=25).apply(lambda s: string.capwords(s, ' '))
    df['date'] = df.date.dt.strftime('%Y-%m-%d')
    return df[['est_id', 'name', 'date', 'grade', 'final_score', 'latlon']].rename(columns={'date': 'latest_inspection'})

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="est_table" 
                        arg-pattern="{{pattern}}" 
                        result="{{est_table}}" 
                        limit="1600 "
                        delay="500" 
                        auto></urth-core-dataframe>
    <urth-viz-table datarows="{{ est_table.data }}" rows-visible="19" selection="{{est_selection}}" columns="{{ est_table.columns }}" selection-as-object>
    </urth-viz-table>
</template>

## Establishment Location

In [ ]:
%%html
<link rel="import" href="urth_components/google-map/google-map.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/google-map/google-map-marker.html" is='urth-core-import' package='GoogleWebComponents/google-map'>

In [ ]:
%%html
<template is="urth-core-bind">
    <google-map style="height: 500px" latitude="[[est_selection.latlon.0]]" longitude="[[est_selection.latlon.1]]" zoom="13">
        <google-map-marker latitude="[[est_selection.latlon.0]]" longitude="[[est_selection.latlon.1]]">
        </google-map-marker>
    </google-map>
</template>

## Histogram of Grades

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-bar/urth-viz-bar.html" is='urth-core-import'>
<style is="custom">
p.title {
    text-align: center;
    font-weight: bold;
}
</style>

In [ ]:
def grade_hist(pattern='', est_id:int=None):
    '''
    Build histograms of grades for establishments matching the pattern
    and (optionally) for a specific establishment ID.
    '''
    # op to apply
    op = (lambda df: df.grade
                       .value_counts(normalize=True, dropna=False)
                       .to_frame())
    # apply to rows matching pattern
    agg = op(filter_df(est_ins, pattern))
    if est_id is not None:
        # apply to rows for the establishment
        sel = op(est_ins[est_ins.est_id == est_id])
        # merge the two series
        agg = pd.merge(agg, sel, how='outer', left_index=True, right_index=True, suffixes=('_all', '_selected'))
    # replace na in the index labels and make the index a column
    return agg.rename(index={np.nan: 'N/A'}).reset_index()

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="grade_hist" 
                        result="{{grade_hist}}" 
                        arg-pattern="{{pattern}}" 
                        arg-est_id="{{est_selection.est_id}}" 
                        delay="500" 
                        auto></urth-core-function>
    <p class="title">All time grades</p>
    <urth-viz-bar datarows='{{grade_hist.data}}' 
                  columns='{{grade_hist.columns}}' 
                  xlabel="Grade" 
                  ylabel="%" 
                  ybounds='[0, 1.0]'></urth-viz-bar>
</template>

## Histogram of Scores

In [ ]:
def score_hist(pattern='', est_id:int=None):
    '''
    Build histograms of scores for establishments matching the pattern
    and (optionally) for a specific establishment ID.
    '''
    # only count grades for full inspections
    df = est_ins[est_ins.type_ins.str.contains('INSPECTION')]
    # op to apply
    op = (lambda df: df.final_score
                       .dropna()
                       .value_counts(normalize=True)
                       .to_frame())
    # apply to rows matching pattern
    agg = op(filter_df(df, pattern))
    if est_id is not None:
        # apply to rows for the establishment
        sel = op(df[df.est_id == est_id])
        # merge the two series
        agg = pd.merge(agg, sel, how='outer', left_index=True, right_index=True, suffixes=('_all', '_selected'))
    # make the index a column and sort by the score
    agg = agg.reset_index().sort_values('index', ascending=False)
    # only show scores over 70 else noise
    return agg[agg['index'] > 70]

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="score_hist" result="{{df}}" arg-pattern="{{pattern}}" arg-est_id="{{est_selection.est_id}}" delay="500" auto></urth-core-function>
    <p class="title">All time scores</p>
    <urth-viz-bar datarows='{{df.data}}' columns='{{df.columns}}' xlabel="Score" ylabel="%" ybounds='[0, 1.0]'></urth-viz-bar>
</template>

## Mean Score Over Time

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-scatter/urth-viz-scatter.html" is='urth-core-import'>

In [ ]:
from datetime import datetime

In [ ]:
def mean_score(period='1q', pattern='', est_id:int=None):
    '''
    Build time series of mean scores per quarter for establishments
    matching the pattern and (optionally) for a specific establishment ID.
    '''
    # op to apply
    op = (lambda df: df.set_index('date')
                       .sort_index()
                       .final_score
                       .resample(period, how='mean').to_frame())
    # apply to rows matching pattern
    agg = op(filter_df(est_ins, pattern))
    if est_id is not None:
        # apply to rows for the establishment
        sel = op(est_ins[est_ins.est_id == est_id])
        # merge the two series
        agg = pd.merge(agg, sel, how='outer', left_index=True, right_index=True, suffixes=('_all', '_selected'))
    # make the index a column
    return agg.reset_index()

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="mean_score" 
                        arg-pattern="{{pattern}}" 
                        result="{{mean_score}}" 
                        arg-est_id="{{est_selection.est_id}}" 
                        delay="500" 
                        auto></urth-core-function>
    
    <p class="title">Mean score per quarter</p>
    <urth-viz-scatter palette='["#4e73ac", "#bc5a54"]' 
                      datarows='{{mean_score.data}}' 
                      rotatelabels="-25" 
                      columns='{{mean_score.columns}}'
                      ybounds='[0, 102]'>
        <urth-viz-col index="0" type="date" format="%Y-%m-%d"></urth-viz-col>
    </urth-viz-scatter>
</template>

## Violations Over Time

In [ ]:
def violation_count(period='1q', pattern='', est_id:int=None):
    '''
    Build time series of mean scores per quarter for establishments
    matching the pattern and (optionally) for a specific establishment ID.
    '''
    # op to apply
    op = (lambda df: df.groupby('date')
                       .count()
                       .sort_index()
                       .resample('1q', how='median')
                       .comments
                       .to_frame())
    # apply to rows matching pattern
    agg = op(filter_df(est_ins_vio, pattern))
    if est_id is not None:
        # apply to rows for the establishment
        sel = op(est_ins_vio[est_ins_vio.est_id == est_id])
        # merge the two series
        agg = pd.merge(agg, sel, how='outer', left_index=True, right_index=True, suffixes=('_all', '_selected'))
    # make the index a column
    return agg.reset_index()

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="violation_count" 
                        result="{{violation_count}}" 
                        arg-pattern="{{pattern}}" 
                        arg-est_id="{{est_selection.est_id}}" 
                        auto></urth-core-function>

    <p class="title">Median violations per quarter</p>
    <urth-viz-scatter datarows='{{violation_count.data}}' 
                      palette='["#4e73ac", "#bc5a54"]'
                      rotatelabels="-25" 
                      ybounds="[0]"
                      columns='{{violation_count.columns}}'>
        <urth-viz-col index="0" type="date" format="%Y-%m-%d"></urth-viz-col>
    </urth-viz-scatter>
</template>

In [ ]:
%%html
<urth-core-channel debug>